# Getting into the flow of TensorFlow

Jupyter notebooks are simply runtime blocks of python code that can interact with eachother. It's a great tool if you're just trying some stuff out with python, you're trying to document a process, or you want an easy to use and visual system to run your python code.

They are composed of markdown blocks and code blocks. This allows for easy stylized documentation.

Code blocks are executed with the "play" button to the left of the block where the brackets are. Once a block is executed, the output stays in memory and any prints, graphs, images, or tables are saved to the notebook itself. This means that a variable or function defined in another block will be visible to all blocks once the block has been executed. This is both useful and can be a pain since everything that's outside of a function global, as if this is one large python script.

You can always rerun a block of code if you're changed things. The outputs will be overwritten and related variables updated.

## Importing libraries

Libraries can be imported anywhere within your code, but often times, people do so at the top of script so that you have everything you need when working.

You can import libraries as is with a simple `import library`. But often it's useful to abreviate so that you don't need to type out the full name like `import lib`. You can also import a function or sub-library like:
`import lib.sublib as slib` or `from lib.sublib import function as func` which will let you use `func()` anwhere in your code then.

In [2]:
import tensorflow as tf

## Datasets


When doing machine learning, you need a dataset to work with for training and testing. This can be images, text, datapoints in some format like csv, etc. In this case, we will be importing the MNIST handwriting dataset where we will try to build a recognizer that will recognize different handwritten numbers as their respective number.

We first import the data set and then during the load step, we get a split of the data with training samples and testing samples. This is normally an 80/20 split.

The X matrix is composed of "features" which describe the data and the Y matrix (or vector) is composed of "labels" which describe what feature we are trying to predict.

In this case, we are working with images that have been converted to matrices of pixels. By default, the values are white color values from 0 to 255 where 0 is black and 255 is white. Everything in between is a form of grey.

We load the data into X and Y training sets, both are subsets of the full set. We also get an X and Y test set which is a smaller subset of the full set. Both of these sets are independent, they have no overlapping samples which is key to measuring the performance of your model (machine learning system).

We then normalize the color values to be floats of range [0., 1.], allowing tensorflow to work with the data.

In [5]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [7]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [8]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.2821123 ,  0.7371514 ,  0.29135478, -0.19054383, -0.68033814,
         0.35191002, -0.2806174 ,  0.39857665, -0.47202754,  0.31637943]],
      dtype=float32)

In [9]:
tf.nn.softmax(predictions).numpy()

array([[0.11284699, 0.17787337, 0.11389481, 0.07034254, 0.04310256,
        0.12100483, 0.06428351, 0.12678556, 0.05308492, 0.11678095]],
      dtype=float32)

In [10]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [11]:
loss_fn(y_train[:1], predictions).numpy()

2.1119246

In [12]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [13]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2961 - accuracy: 0.9145
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1428 - accuracy: 0.9569
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1089 - accuracy: 0.9666
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0883 - accuracy: 0.9721
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0757 - accuracy: 0.9757


In [14]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0775 - accuracy: 0.9765


[0.07753873616456985, 0.9764999747276306]

In [16]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [17]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[4.94272285e-07, 1.75127568e-09, 2.65360632e-05, 2.52797152e-03,
        2.34537606e-12, 2.12764076e-06, 1.70575793e-13, 9.97421265e-01,
        1.23622704e-06, 2.03926829e-05],
       [2.13027018e-09, 7.55577523e-04, 9.99213696e-01, 2.48586875e-05,
        9.44303771e-13, 3.59221349e-06, 1.00196884e-09, 3.05898238e-13,
        2.28463705e-06, 7.20609614e-11],
       [1.77568978e-07, 9.99027252e-01, 1.48744206e-04, 9.00996474e-06,
        1.37456009e-04, 5.33107709e-07, 4.82036703e-05, 4.42492805e-04,
        1.84798540e-04, 1.50466974e-06],
       [9.99963999e-01, 1.07140785e-08, 3.14118624e-05, 5.41694419e-07,
        1.27185764e-08, 5.88358773e-07, 7.13899766e-08, 1.24362327e-06,
        1.69997316e-09, 2.15564182e-06],
       [1.17742538e-05, 4.67175765e-09, 3.66559539e-06, 3.61286681e-08,
        9.96726394e-01, 1.45428672e-07, 1.60380641e-05, 5.33738130e-05,
        3.60685419e-07, 3.18811601e-03]], dtype=float32)>